In [19]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [20]:
!pip install textsearch
!pip install contractions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
categories = ['airlines', 'celebrities', 'colleges', 'fastfood', 
              'leagues', 'news', 'streamingplatforms', 'techgiants']

subCategories = [['output_AmericanAir', 'output_JetBlue', 'output_qatarairways', 'output_SingaporeAir'], 
                 ['output_TheRock', 'output_ArianaGrande', 'output_chrissyteigen', 'output_jimmyfallon'], 
                 ['output_Stanford', 'output_Harvard', 'output_MIT', 'output_UCBerkeley'], 
                 ['output_McDonalds', 'output_BurgerKing', 'output_Starbucks', 'output_wendys'], 
                 ['output_IPL', 'output_MLB', 'output_MLS', 'output_NBA'], 
                 ['output_BBCWorld', 'output_CNN', 'output_washingtonpost', 'output_nytimes'], 
                 ['output_Netflix', 'output_Spotify', 'output_PrimeVideo', 'output_HBO'], 
                 ['output_amazon', 'output_facebook', 'output_Tesla', 'output_Microsoft']]

In [22]:
sizeOfSets = 400

categCount = 0
for i in range(len(categories)):
    for j in subCategories[i]:
        categCount += 1
        
categCount *= sizeOfSets

val = [None] * categCount # Keep value (sizeOfSets * no. of categories)
sizeOfTweetContent = 75

print(len(val)) # No. Of Posts

12800


In [23]:
def inputDataInDataframe():
    overCount= 0
    for i in range(len(categories)):
        for j in subCategories[i]:
            direc:str = '/Users/sameernawaz/AnacondaProjects/Datasets/tweets/'+categories[i]+'/'+j+'.csv'
            with open(direc, 'r') as csv_file:
                csv_reader = csv.reader(csv_file)
                count = 0
                for line in csv_reader:
                    if len(line[6]) > sizeOfTweetContent:
                        val[overCount] = line
                        count += 1
                        overCount += 1
                        if count >= sizeOfSets :
                            break

In [24]:
inputDataInDataframe()

In [25]:
df = pd.DataFrame(val, columns = ['date', 'username', 'to', 'replies', 
                                  'retweets', 'favorites', 'text', 'geo', 
                                  'mentions', 'hashtags', 'id', 'permalink']) 

In [26]:
# Once data added in dataframe, Shuffle it
df = df.sample(frac=1).reset_index(drop=True)
# Add a unique id (rec_id) for all tweets
df = df.assign(rec_id=np.arange(len(df))).reset_index(drop=True)

In [27]:
df = df[['rec_id', 'username', 'text']]
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12800 entries, 0 to 12799
Data columns (total 3 columns):
rec_id      12800 non-null int64
username    12800 non-null object
text        12800 non-null object
dtypes: int64(1), object(2)
memory usage: 400.0+ KB


In [28]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['text']))
len(norm_corpus)

12800

In [29]:
# Extract TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(12800, 26984)

In [30]:
# Compute Pairwise Document Similarity

from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,12790,12791,12792,12793,12794,12795,12796,12797,12798,12799
0,1.000000,0.013898,0.052041,0.025284,0.0,0.0,0.000000,0.0,0.000000,0.014855,...,0.0,0.009271,0.0,0.042221,0.025714,0.000000,0.0,0.008753,0.0,0.000000
1,0.013898,1.000000,0.013952,0.000000,0.0,0.0,0.000000,0.0,0.029604,0.000000,...,0.0,0.013721,0.0,0.351401,0.000000,0.013997,0.0,0.012955,0.0,0.019976
2,0.052041,0.013952,1.000000,0.010113,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.012198,0.0,0.008266,0.018910,0.000000,0.0,0.000000,0.0,0.009926
3,0.025284,0.000000,0.010113,1.000000,0.0,0.0,0.041232,0.0,0.000000,0.038237,...,0.0,0.013577,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
4,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000


In [31]:
# Get list of post IDs

posts_list = df['rec_id'].values
posts_list, posts_list.shape

(array([    0,     1,     2, ..., 12797, 12798, 12799]), (12800,))

In [32]:
posts_idx = np.where(posts_list == 3997)[0][0]
posts_idx

3997

In [33]:
posts_similarities = doc_sim_df.iloc[posts_idx].values
posts_similarities

array([0., 0., 0., ..., 0., 0., 0.])

In [34]:
# Get top 5 similar post IDs

similar_posts_idxs = np.argsort(-posts_similarities)[1:6]
similar_posts_idxs

array([12250, 10236,  9883,  6841,  1460])

In [35]:
similar_posts = posts_list[similar_posts_idxs]
similar_posts

array([12250, 10236,  9883,  6841,  1460])

In [36]:
def relevant_posts(postId, posts = posts_list, doc_sims = doc_sim_df):
    # find post id
    posts_idx = np.where(posts == postId)[0][0]
    # get posts similarities
    posts_similarities = doc_sims.iloc[posts_idx].values
    # get top 5 similar post IDs
    similar_posts_idxs = np.argsort(-posts_similarities)[1:6]
    # get top 5 posts
    similar_posts = posts[similar_posts_idxs]
    # return the top 5 posts
    return similar_posts

In [38]:
post_id = 1059

print('-----Input Post-----')
print((df.loc[df['rec_id'] == post_id]['text']).values[0])
print('\n-----Relevant Posts-----')
for i in relevant_posts(postId = post_id, posts = posts_list, doc_sims = doc_sim_df):
    print("=>",(df.loc[df['rec_id'] == i]['text']).values[0])

-----Input Post-----
An algorithm developed at @MIT_CSAIL could help doctors better analyze the placenta and monitor for potential issues during pregnancy. via @engadget

-----Relevant Posts-----
=> An algorithm developed to study the structure of galaxies helps explain a key feature of embryonic development. http://mitsha.re/SgsM50vjL4N
=> The #HoloLens2 is better than ever and is now shipping to customers. Get to know the mixed reality headset: http://msft.social/pqlAYR via @engadget
=> Mechanical engineers have developed a steerable, thread-like robot that can actively glide through narrow, winding blood vessels. This robotic thread could help doctors treat blockages and lesions that occur in aneurysms and stroke. Video by Melanie Gonick/MIT
=> #LegenDMB @DaMarcusBeasley wants to help the potential DaMarcus Beasleys of tomorrow.
=> This advance may one day help doctors treat patients with traumatic #brain injuries, strokes and aneurysms. #BerkeleyResearch
